In [4]:
! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0  2152k      0 --:--:-- --:--:-- --:--:-- 2156k


In [5]:
import zipfile

with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
  zip_ref.extractall('data')

In [6]:
import pandas as pd

movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')


In [7]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)


The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [8]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [10]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")


Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [11]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [2]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [12]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: False
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0278, 0.0249, 0.0418,  ..., 0.0046, 0.0045, 0.0402],
        [0.0184, 0.0254, 0.0129,  ..., 0.0171, 0.0230, 0.0027],
        [0.0199, 0.0124, 0.0297,  ..., 0.0057, 0.0406, 0.0476],
        ...,
        [0.0247, 0.0418, 0.0240,  ..., 0.0094, 0.0171, 0.0039],
        [0.0417, 0.0320, 0.0412,  ..., 0.0027, 0.0094, 0.0367],
        [0.0401, 0.0426, 0.0181,  ..., 0.0030, 0.0294, 0.0269]])
item_factors.weight tensor([[0.0073, 0.0280, 0.0324,  ..., 0.0037, 0.0361, 0.0301],
        [0.0307, 0.0079, 0.0085,  ..., 0.0131, 0.0301, 0.0032],
        [0.0368, 0.0429, 0.0335,  ..., 0.0027, 0.0191, 0.0036],
        ...,
        [0.0356, 0.0037, 0.0336,  ..., 0.0362, 0.0489, 0.0218],
        [0.0356, 0.0259, 0.0324,  ..., 0.0096, 0.0305, 0.0453],
        [0.0448, 0.0127, 0.0406,  ..., 0.0253, 0.0103, 0.0362]])


In [14]:
from tqdm.notebook import tqdm

for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
        if cuda:
            x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        outputs = model(x)
        loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.063586647135352
iter #1 Loss: 4.739511869885595
iter #2 Loss: 2.473414234249725
iter #3 Loss: 1.7212224935517093
iter #4 Loss: 1.3461341383644772
iter #5 Loss: 1.1290153749098026
iter #6 Loss: 0.991758400111029
iter #7 Loss: 0.9006674475778783
iter #8 Loss: 0.8375220277587775
iter #9 Loss: 0.7925770602704305
iter #10 Loss: 0.7598078544369808
iter #11 Loss: 0.7347062657856699
iter #12 Loss: 0.7160312970580184
iter #13 Loss: 0.7018711582431333
iter #14 Loss: 0.690606141075265
iter #15 Loss: 0.6818203602723664
iter #16 Loss: 0.6748524039681187
iter #17 Loss: 0.669752202512044
iter #18 Loss: 0.6657579118754658
iter #19 Loss: 0.6628395955877255
iter #20 Loss: 0.6609171656560777
iter #21 Loss: 0.6590239557669247
iter #22 Loss: 0.6575696916522713
iter #23 Loss: 0.656632874520297
iter #24 Loss: 0.6560629034405432
iter #25 Loss: 0.6551372926625504
iter #26 Loss: 0.6540912528933608
iter #27 Loss: 0.6537797086054299
iter #28 Loss: 0.6526894000581074
iter #29 Loss: 0.6516008721617

In [16]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.7485,  1.0311,  0.6971,  ...,  1.1096,  0.8273,  1.4733],
        [ 1.3846,  0.9662,  0.2484,  ...,  0.4563,  1.0697,  1.9260],
        [ 0.3999,  2.7182, -1.0256,  ..., -1.1149,  2.6475, -0.8333],
        ...,
        [ 1.3045,  2.0183,  2.2333,  ..., -0.2981,  0.8710, -0.4052],
        [ 0.6569,  1.2991,  1.1398,  ...,  0.8815,  0.6948,  1.2061],
        [ 1.7662,  1.0654, -0.0072,  ...,  1.3880,  1.3884,  1.0412]])
item_factors.weight tensor([[ 0.4113,  0.3169,  0.3935,  ...,  0.7557,  0.6205,  0.7705],
        [ 0.3055,  0.6547, -0.3038,  ...,  0.4010,  0.3057,  0.7952],
        [ 0.6868,  0.7421,  0.3299,  ...,  0.6752,  0.4426,  0.5093],
        ...,
        [ 0.3489,  0.3169,  0.2741,  ...,  0.3495,  0.3627,  0.3355],
        [ 0.4215,  0.4103,  0.4093,  ...,  0.3948,  0.4156,  0.4297],
        [ 0.3999,  0.3676,  0.4092,  ...,  0.3782,  0.3644,  0.3909]])


In [17]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()


In [18]:
len(trained_movie_embeddings) # unique movie factor weights


9724

In [19]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [20]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])


Cluster #0
	 Terminator 2: Judgment Day (1991)
	 Lord of the Rings: The Return of the King, The (2003)
	 Sixth Sense, The (1999)
	 Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
	 Dances with Wolves (1990)
	 Alien (1979)
	 Terminator, The (1984)
	 Godfather: Part II, The (1974)
	 Babe (1995)
	 Blade Runner (1982)
Cluster #1
	 Coneheads (1993)
	 Batman & Robin (1997)
	 Free Willy (1993)
	 Godzilla (1998)
	 Anaconda (1997)
	 Little Nicky (2000)
	 Toys (1992)
	 Honey, I Blew Up the Kid (1992)
	 Barb Wire (1996)
	 Speed 2: Cruise Control (1997)
Cluster #2
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Matrix, The (1999)
	 Star Wars: Episode IV - A New Hope (1977)
	 Toy Story (1995)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 Lord of the Rings: The Fellowship of the Ring, The (2001)
	 Star Wars: Episode VI - Return of the Jedi (1983)
Cluster #3
	 Schindler